In [ ]:
# Start an XVIZ server on port 3002
from streetscape_avs import SimpleServer
xvizServer = SimpleServer(port=3002)
live_server = await xvizServer.serve()

In [ ]:
# If needed we can close the server to restart
# live_server.close()

In [ ]:
# Generate a Solar system in XVIZ
"""
This module provides a example scenario where a vehicle drives along a circle.
"""

import time
import math
from random import randint, uniform


import xviz_avs as xviz
import xviz_avs.builder as builder
import xviz_avs.io as io

# This demonstrates the use of a Scene Graph to manage spatial relationships
# between XVIZ data.
# 
# Instead of defining everything in a single coordinate system
# this generator will use XVIZ Scene Graph support. The Scene Graph is defined in XVIZ
# using the *pose* and *link* methods in the builder.  They *pose* defines the transformation,
# here using position and orientation, between nodes.  The *link* will create the relationship
# between poses and primitives. The affective transform of the primitives is the concatenation
# of all the defined transform in the path for a given primitive.
# 
# For example, below there is a path /system -> /earth/orbit -> /earth/pose -> /earth/body
# and the poses along that path are multipled to create the transformation that is applied to the
# primitive data in the stream /earth/body
class SolarSystemGenerator:
    def __init__(self):
        self.sun = None
        self.earth = None
        self.mars = None
        self.moon = None
        
    def _drawPose(self, timestamp, name, distance, degreesPerSecond, pitch=0):
        angle = math.radians(timestamp * degreesPerSecond)
        self.builder.pose(name)\
          .position(distance, 0, 0)\
          .orientation(pitch, 0, angle)

    def _drawPoses(self, timestamp):
        self._drawPose(timestamp, '/system', 0, 0) 
        self._drawPose(timestamp, '/earth/orbit', 0, 45, 0.1) 
        self._drawPose(timestamp, '/moon/orbit', 0, 45, 0.3) 
        self._drawPose(timestamp, '/mars/orbit', 0, -45, -0.2) 
        self._drawPose(timestamp, '/sun/pose', 0, 45) 
        self._drawPose(timestamp, '/earth/pose', 25, 45) 
        self._drawOrbit('earth', 25)
        self._drawPose(timestamp, '/moon/pose', 10, 45) 
        self._drawOrbit('moon', 10)
        self._drawPose(timestamp, '/mars/pose', 45, 45)
        self._drawOrbit('mars', 45)
  
    def _drawLinks(self, timestamp):
        # TODO(twojtasz): Currently these are being generated on every "frame" but once
        # the python XVIZ library supports the 'PERSISTENT' option, they can be sent only once
        self.builder.link('/system', '/sun/pose')
        self.builder.link('/system', '/earth/orbit')
        self.builder.link('/system', '/mars/orbit')
        self.builder.link('/sun/pose', '/sun/body')
        self.builder.link('/earth/orbit', '/earth/orbit/mark')
        self.builder.link('/earth/orbit', '/earth/pose')
        self.builder.link('/earth/pose', '/earth/body')
        self.builder.link('/earth/pose', '/moon/orbit')
        self.builder.link('/moon/orbit', '/moon/pose')
        self.builder.link('/moon/orbit', '/moon/orbit/mark')
        self.builder.link('/moon/pose', '/moon/body')
        self.builder.link('/mars/orbit', '/mars/pose')
        self.builder.link('/mars/orbit', '/mars/orbit/mark')
        self.builder.link('/mars/pose', '/mars/body')

    def _drawBody(self, name, pts):
        self.builder.primitive(name)\
            .points(pts)\
            .id(name)

    def _drawOrbit(self, name, radius):
        # TODO(twojtasz): Make this 'PERSISTENT'
        numPoints = radius * 3
        anglePerPoint = math.radians(360/numPoints)
        for p in range(0, numPoints):
            angle = p * anglePerPoint
            self.builder.primitive('/'+name+'/orbit/mark').circle([radius*math.cos(angle), radius*math.sin(angle), 0], 0.5)

    def _createBody(self, r):
        pts = []
        for p in range(0, r*100):
            theta = uniform(0, math.pi)
            gamma = uniform(0, math.tau)
            x = r * math.sin(theta)*math.cos(gamma)
            y = r * math.sin(theta)*math.sin(gamma)
            z = r * math.cos(theta)

            pts.extend([x, y, z])
        return pts
    
    def _createBodies(self):
        # TODO(twojtasz): Make this 'PERSISTENT'
        if self.sun:
            return
        
        self.sun = self._createBody(9)
        self.earth = self._createBody(5)
        self.moon = self._createBody(2)
        self.mars = self._createBody(4)

    def _drawBodies(self):
        self._createBodies()
        self._drawBody('/sun/body', self.sun)
        self._drawBody('/earth/body', self.earth)
        self._drawBody('/moon/body', self.moon)
        self._drawBody('/mars/body', self.mars)

    def _generate_xviz(self, start, end):
        # MemorySource is just a dictionary that conforms to the IO interface XVIZ expects
        store = io.MemorySource()
        
        # XVIZJsonWriter will output JSON XVIZ data
        # TODO(twojtasz): Get the XVIZProtobufWriter working
        writer = io.XVIZJsonWriter(store)
        
        # The metadata defines the streams and attribute for them up front
        meta = self.get_metadata(start, end)
        writer.write_message(meta)
        
        # Below we loop until we reach the 'end' time, with steps every 0.1 seconds
        step = 0.1 # 10hz
        current = start
        while current < end:
            # Create a builder to construct the XVIZ data
            self.builder = xviz.XVIZBuilder()
            
            delta = current - start

            
            self._drawBodies()
            self._drawLinks(delta)
            self._drawPoses(delta)
            
            # TODO(twojtasz): Current python XVIZ library forces the
            # default pose stream '/vehicle_pose' to be defined
            self.builder.pose().position(0, 0, 0).timestamp(current)
            
            writer.write_message(self.builder.get_message())

            current += step
        return store

    def generate(self, duration):
        start = time.time()
        return self._generate_xviz(start, start+duration)
        
    def get_metadata(self, start, end):
        # 
        builder = xviz.XVIZMetadataBuilder()
        builder.start_time(start)
        builder.end_time(end)
       
        builder.stream("/vehicle_pose").category(xviz.CATEGORY.POSE)
        builder.stream("/system").category(xviz.CATEGORY.POSE)
        builder.stream("/sun/pose").category(xviz.CATEGORY.POSE)
        builder.stream("/earth/orbit").category(xviz.CATEGORY.POSE)
        builder.stream("/mars/orbit").category(xviz.CATEGORY.POSE)
        builder.stream("/moon/orbit").category(xviz.CATEGORY.POSE)
        builder.stream("/earth/pose").category(xviz.CATEGORY.POSE)
        builder.stream("/moon/pose").category(xviz.CATEGORY.POSE)
        builder.stream("/mars/pose").category(xviz.CATEGORY.POSE)

        builder.stream("/sun/body").type(xviz.PRIMITIVE_TYPES.POINT).stream_style({'fill_color': [255, 255, 51, 200], 'radius_pixels': 7})
        builder.stream("/earth/body").type(xviz.PRIMITIVE_TYPES.POINT).stream_style({'fill_color': [0, 200, 0, 200], 'radius_pixels': 3})
        builder.stream("/moon/body").type(xviz.PRIMITIVE_TYPES.POINT).stream_style({'fill_color': [50, 50, 50, 200], 'radius_pixels': 3})
        builder.stream("/mars/body").type(xviz.PRIMITIVE_TYPES.POINT).stream_style({'fill_color': [150, 0, 0, 200], 'radius_pixels': 3})
        builder.stream("/earth/orbit/mark").type(xviz.PRIMITIVE_TYPES.CIRCLE).stream_style({'fill_color': [0, 0, 70, 128], 'stroked': False})
        builder.stream("/moon/orbit/mark").type(xviz.PRIMITIVE_TYPES.CIRCLE).stream_style({'fill_color': [0, 0, 70, 128], 'stroked': False})
        builder.stream("/mars/orbit/mark").type(xviz.PRIMITIVE_TYPES.CIRCLE).stream_style({'fill_color': [0, 0, 70, 128], 'stroked': False})

        return builder.get_message()

In [ ]:
%%capture
# Note: %%capture is used because a verbose warning will be printed otherwise

# Create instance of simulation generator
runner = SolarSystemGenerator()

# Generate simulation and register with server with log id 'default'
xvizServer.register('default', runner.generate(10))

In [ ]:
# Use the streetscape XVIZ viewer to view the data
from streetscape_avs import Streetscape
widget = Streetscape(log='default', port=3002)
widget